In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report, accuracy_score

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_df = pd.read_csv(Path("Resources/lending_data.csv"))
# Review the DataFrame
lending_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_df["loan_status"]
# Separate the X variable, the features
X = lending_df.drop(columns="loan_status", axis=1)

# y.nunique()

In [4]:
# Review the y variable Series
print(y[:5], y.shape)

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64 (77536,)


In [5]:
# Review the X variable DataFrame
print(X[:5], X.shape)

   loan_size  interest_rate  borrower_income  debt_to_income  num_of_accounts  \
0    10700.0          7.672            52800        0.431818                5   
1     8400.0          6.692            43600        0.311927                3   
2     9000.0          6.963            46100        0.349241                3   
3    10700.0          7.664            52700        0.430740                5   
4    10800.0          7.698            53000        0.433962                5   

   derogatory_marks  total_debt  
0                 1       22800  
1                 0       13600  
2                 0       16100  
3                 1       22700  
4                 1       23000   (77536, 7)


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y_value_count = y.value_counts()
y_value_count

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lr_model = LogisticRegression(solver='lbfgs', max_iter=500, random_state=1)
# Fit the model using training data
lr_model.fit(X_train, y_train)

LogisticRegression(max_iter=500, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
training_prediction = lr_model.predict(X_train)
test_prediction = lr_model.predict(X_test)
# result = pd.DataFrame({"Predictions": prediction, "Actual": y_test}).reset_index(drop=True)
# result.head(10)
max(lr_model.n_iter_)

for x in lr_model.n_iter_:
    print(x)

33


In [10]:
from sklearn import __version__
print(__version__)

1.0.2


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [11]:
# Print the balanced_accuracy score of the model
print(f"The balanced accuracy score of the model is: {balanced_accuracy_score(y_test, test_prediction)}")
print(accuracy_score(y_test, test_prediction))

The balanced accuracy score of the model is: 0.9520479254722232
0.9918489475856377


In [12]:
# Generate a confusion matrix for the model
training_matrix = confusion_matrix(y_train, training_prediction)
test_matrix = confusion_matrix(y_test, test_prediction)
print("Train Matrix:")
print(training_matrix)
print("Test Matrix:")
print(test_matrix)

Train Matrix:
[[55994   277]
 [  181  1700]]
Test Matrix:
[[18663   102]
 [   56   563]]


In [13]:
# Print the classification report for the model
train_classification_report = classification_report(y_train, training_prediction)
print("Train Classification Report:")
print(train_classification_report)
test_classification_report = classification_report(y_test, test_prediction)
print("Test Classification Report")
print(test_classification_report)

Train Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56271
           1       0.86      0.90      0.88      1881

    accuracy                           0.99     58152
   macro avg       0.93      0.95      0.94     58152
weighted avg       0.99      0.99      0.99     58152

Test Classification Report
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Based on the Confusion Matrices and the Classification Reports, we can say, the logistic regression model performed well in predicting both the 0 (healthy loan) and 1 (high-risk loan) labels. The model achieved an f1-score of 1.00 in predicting the 0 label, indicating perfect precision and recall. For the 1 label, the model achieved an f1-score of 0.88, indicating good performance but slightly lower than that of the 0 label.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [14]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
oversampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
X_test_resampled, y_test_resampled = oversampler.fit_resample(X_test, y_test)

In [20]:
# Count the distinct values of the resampled labels data
unique_labels_tr = np.unique(X_train_resampled)
unique_labels_te = np.unique(X_test_resampled)
print("Number of distinct values in y_train_resampled:", len(unique_labels_tr))
print("Number of distinct values in y_test_resampled:", len(unique_labels_te))


Number of distinct values in y_train_resampled: 5980
Number of distinct values in y_test_resampled: 4527


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [16]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lr_model_2 = LogisticRegression(solver='lbfgs', max_iter=500, random_state=1)
# Fit the model using the resampled training data
lr_model_2.fit(X_train_resampled, y_train_resampled)
# Make a prediction using the testing data
training_prediction_2 = lr_model_2.predict(X_train_resampled)
test_prediction_2 = lr_model_2.predict(X_test_resampled)


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [17]:
# Print the balanced_accuracy score of the model 
print(f"The balanced accuracy score of the test model is: {balanced_accuracy_score(y_test_resampled, test_prediction_2)}")
print(f"The accuracy score of the test model is: {accuracy_score(y_test_resampled, test_prediction_2)}")

The balanced accuracy score of the test model is: 0.9938715694111377
The accuracy score of the test model is: 0.9938715694111377


In [18]:
# Generate a confusion matrix for the model
training_matrix_2 = confusion_matrix(y_train_resampled, training_prediction_2)
test_matrix_2 = confusion_matrix(y_test_resampled, test_prediction_2)
print("Train Matrix:")
print(training_matrix_2)
print("Test Matrix:")
print(test_matrix_2)

Train Matrix:
[[55964   307]
 [  286 55985]]
Test Matrix:
[[18649   116]
 [  114 18651]]


In [19]:
# Print the classification report for the model
train_classification_report_2 = classification_report(y_train_resampled, training_prediction_2)
print("Train Classification Report:")
print(train_classification_report_2)
test_classification_report_2 = classification_report(y_test_resampled, test_prediction_2)
print("Test Classification Report")
print(test_classification_report_2)

Train Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     56271
           1       0.99      0.99      0.99     56271

    accuracy                           0.99    112542
   macro avg       0.99      0.99      0.99    112542
weighted avg       0.99      0.99      0.99    112542

Test Classification Report
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     18765
           1       0.99      0.99      0.99     18765

    accuracy                           0.99     37530
   macro avg       0.99      0.99      0.99     37530
weighted avg       0.99      0.99      0.99     37530



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model, trained with oversampled data, performed well in predicting both the `0` (healthy loan) and `1` (high-risk loan) labels, as evidenced by the high (`0.99`) precision, recall, and f1-scores for both classes in the training and testing classification reports. The confusion matrices show that the model predicted the majority of both 0 and 1 labels correctly in both the training and testing sets. 

Additionally, compared to the model trained on non-oversampled data, the oversampled model performed better in predicting high-risk loans, indicating its effectiveness in mitigating lending risks.